In [4]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D
import numpy as np
import time

def create_new_model():
    IMAGE_SIZE = [32, 32]
    vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    prediction = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=vgg.input, outputs=prediction)
    
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

start = time.time()

model_glaucoma = create_new_model()
model_glaucoma.load_weights("gsaved_models/model_2.h5")

model_normal = create_new_model()
model_normal.load_weights("nsaved_models/model_5.h5")

model_other = create_new_model()
model_other.load_weights("osaved_models/model_2.h5")

test_datagen = ImageDataGenerator(rescale=1. / 255)


test_generator = test_datagen.flow_from_directory(
    directory="test_images",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_generator.reset()

list_ans = []

for i in range(STEP_SIZE_TEST):
    test = next(test_generator)
    pred1 = model_glaucoma.predict(test,verbose=0)
    pred1 = np.transpose(pred1)[0]
    pred1 = list(map(lambda x: 0 if x < 0.1 else 1, pred1)) 
    if np.array_equal(pred1,np.array([0])) :
        pred2 = model_normal.predict(test,verbose=0)
        pred2 = np.transpose(pred2)[0]
        pred2 = list(map(lambda x: 0 if x < 0.5 else 1, pred2))
        if np.array_equal(pred2,np.array([0])) :
            pred3 = model_normal.predict(test,verbose=0)
            pred3 = np.transpose(pred3)[0]
            pred3 = list(map(lambda x: 0 if x < 0.5 else 1, pred3))
            if np.array_equal(pred3,np.array([0])) :
                list_ans.append('Normal')
            else:
                list_ans.append('Other')
        else:
            list_ans.append('Normal')
    else:
        list_ans.append('glaucoma')
        
end = time.time()

Found 5 images belonging to 2 classes.


In [5]:
print(f"Runtime of the program is {end - start}")
print(list_ans)

Runtime of the program is 1.6349976062774658
['Normal', 'glaucoma', 'glaucoma', 'Normal', 'glaucoma']


In [ ]:
print(len(list_ans))

In [14]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D
import numpy as np
import time

def create_new_model():
    IMAGE_SIZE = [224, 224]
    vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    prediction = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=vgg.input, outputs=prediction)
    
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

start = time.time()

model_normal = create_new_model()
model_normal.load_weights("nsaved_models/model_4.h5")


test_datagen = ImageDataGenerator(rescale=1. / 255)


test_generator = test_datagen.flow_from_directory(
    directory="test_images",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_generator.reset()

list_ans = []

pred1 = model_normal.predict(test_generator,
                    steps=STEP_SIZE_TEST,
                    verbose=1)
print(pred1)
pred1 = np.transpose(pred1)[0]
pred1 = list(map(lambda x: 0 if x > 0.5 else 1, pred1))
print(pred1)
    

Found 5 images belonging to 2 classes.
5/5 [==============================] - 5s 1s/step
[[0.7309504 ]
 [0.02926825]
 [0.6354384 ]
 [0.06961735]
 [0.7827863 ]]
[0, 1, 0, 1, 0]
